[homework questions link](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md) 

# question1:
running this on github code spaces
```bash
docker run --rm \
    --name elastic \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
```
and then running this
```bash
curl localhost:9200
```

`version.build_hash` is `dbcbbbd0bc4924cfeb28929dc05d82d662c527b7`

# question2:

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')
index_name = "course-questions"

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [5]:
es_client.count(index=index_name)

ObjectApiResponse({'count': 948, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

# question3:

In [6]:
query = 'How do execute a command on a Kubernetes pod?'
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

res = es_client.search(index=index_name, body=search_query)

In [7]:
list(map(lambda x:x['_score'] , res.body['hits']['hits']))[0]

44.50556

# question4:

In [20]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

res = es_client.search(index=index_name, body=search_query)

In [21]:
list(map(lambda x:x["_source"]["question"], res.body['hits']['hits']))[-1]

'How do I copy files from a different folder into docker container’s working directory?'

# question5:

In [22]:
def generate_context(document: dict):
    return f"""
Q: {document['question']}
A: {document['text']}
    """.strip()

context = "\n\n".join(list(map(lambda x:generate_context(x["_source"]), res.body['hits']['hits'])))
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [23]:
question = "How do copy a file to a Docker container?"
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [24]:
len(prompt_template)

1446

# question6:

In [25]:
import tiktoken


encoding = tiktoken.encoding_for_model("gpt-4o")

len(encoding.encode(prompt_template))

320

# Generating the answer

In [45]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the value of an environment variable
API_TOKEN = os.getenv('HUGGING_FACE_API_TOKEN')

In [81]:
import requests
API_URL = "https://api-inference.huggingface.co/models/google/gemma-1.1-7b-it"
headers = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/json"
}
response = requests.post(API_URL, headers=headers, json={"inputs":prompt_template})

In [82]:
print(response.json()[0]['generated_text'][len(prompt_template):])



Answer:
To execute a command in a running docker container, use the `docker exec -it <container-id> bash` command.
